# 🔍 GradCAM Visualization for CNN
Visualize CNN decision regions for MNIST-C.

In [ ]:
import cv2
import torch
import numpy as np
from torchvision import models, transforms
from matplotlib import pyplot as plt

# Load trained model (assume it's available)
model = torch.load("cnn_mnist_model.pt")
model.eval()

def gradcam(input_tensor, model, target_layer):
    gradients = []
    activations = []

    def backward_hook(module, grad_input, grad_output):
        gradients.append(grad_output[0])

    def forward_hook(module, input, output):
        activations.append(output)

    handle_b = target_layer.register_backward_hook(backward_hook)
    handle_f = target_layer.register_forward_hook(forward_hook)

    output = model(input_tensor)
    target_class = output.argmax().item()
    model.zero_grad()
    class_loss = output[0, target_class]
    class_loss.backward()

    grad = gradients[0][0].detach().numpy()
    act = activations[0][0].detach().numpy()

    weights = grad.mean(axis=(1, 2))
    cam = np.sum(weights[:, None, None] * act, axis=0)
    cam = np.maximum(cam, 0)
    cam = cv2.resize(cam, (28, 28))
    cam -= cam.min()
    cam /= cam.max()

    handle_b.remove()
    handle_f.remove()

    return cam

# Visualize example
# (Assumes `input_tensor` is a preprocessed 1x1x28x28 image)
